In [3]:
from openai import OpenAI

client = OpenAI()



In [4]:
response = client.responses.create(
    model='gpt-5',
    # instructions='You are helpful assistant.',
    input='Hello!'
)

print(response.output_text)

Hi there! How can I help you today?


In [7]:
response = client.responses.create(
    model='gpt-5',
    # instructions='You are helpful assistant.',
    input=[
        {
            'role': 'user',
            'content': 'Hello!'
        }
    ]
)

print(response.model_dump_json())

{"id":"resp_00d22107e85098430068e0c4fe1d688196942360786720c6d2","created_at":1759560958.0,"error":null,"incomplete_details":null,"instructions":null,"metadata":{},"model":"gpt-5-2025-08-07","object":"response","output":[{"id":"rs_00d22107e85098430068e0c4fed9648196ae291978d35cc5ef","summary":[],"type":"reasoning","encrypted_content":null,"status":null},{"id":"msg_00d22107e85098430068e0c4ff8b0481968cceb3382cae4a18","content":[{"annotations":[],"text":"Hi! How can I help you today?","type":"output_text","logprobs":[]}],"role":"assistant","status":"completed","type":"message"}],"parallel_tool_calls":true,"temperature":1.0,"tool_choice":"auto","tools":[],"top_p":1.0,"background":false,"max_output_tokens":null,"max_tool_calls":null,"previous_response_id":null,"prompt":null,"reasoning":{"effort":"medium","generate_summary":null,"summary":null},"service_tier":"default","status":"completed","text":{"format":{"type":"text"},"verbosity":"medium"},"top_logprobs":0,"truncation":"disabled","usage":{

In [2]:
tools = [
    {
        "type": "function",
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia",
                }
            },
            "required": ["location"],
            "additionalProperties": False,
        },
        "strict": True,
    },
]

response = client.responses.create(
    model="gpt-5",
    input=[
        {"role": "user", "content": "What is the weather like in Paris today?"},
    ],
    tools=tools,
)

print(response.output[0].to_json())

{
  "id": "rs_058c621fa5129c3c0068e0003394d08195a976dc12c5035800",
  "summary": [],
  "type": "reasoning"
}


In [3]:
import json

# 1. Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "name": "get_horoscope",
        "description": "Get today's horoscope for an astrological sign.",
        "parameters": {
            "type": "object",
            "properties": {
                "sign": {
                    "type": "string",
                    "description": "An astrological sign like Taurus or Aquarius",
                },
            },
            "required": ["sign"],
        },
    },
]

def get_horoscope(sign):
    return f"{sign}: Next Tuesday you will befriend a baby otter."

# Create a running input list we will add to over time
input_list = [
    {"role": "user", "content": "What is my horoscope? I am an Aquarius."}
]

# 2. Prompt the model with tools defined
response = client.responses.create(
    model="gpt-5",
    tools=tools,
    input=input_list,
)

# Save function call outputs for subsequent requests
input_list += response.output

for item in response.output:
    if item.type == "function_call":
        if item.name == "get_horoscope":
            # 3. Execute the function logic for get_horoscope
            horoscope = get_horoscope(json.loads(item.arguments))
            
            # 4. Provide function call results to the model
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "horoscope": horoscope
                })
            })

print("Final input:")
print(input_list)

response = client.responses.create(
    model="gpt-5",
    instructions="Respond only with a horoscope generated by a tool.",
    tools=tools,
    input=input_list,
)

# 5. The model should be able to give a response!
print("Final output:")
print(response.model_dump_json(indent=2))
print("\n" + response.output_text)

Final input:
[{'role': 'user', 'content': 'What is my horoscope? I am an Aquarius.'}, ResponseReasoningItem(id='rs_0b790d2a6e363c610068e01745c26c8197ba119da7f0b75446', summary=[], type='reasoning', encrypted_content=None, status=None), ResponseFunctionToolCall(arguments='{"sign":"Aquarius"}', call_id='call_gSPLh0nKeSojRJUeKrBusBAs', name='get_horoscope', type='function_call', id='fc_0b790d2a6e363c610068e01747f67081979f4199af956e03a9', status='completed'), {'type': 'function_call_output', 'call_id': 'call_gSPLh0nKeSojRJUeKrBusBAs', 'output': '{"horoscope": "{\'sign\': \'Aquarius\'}: Next Tuesday you will befriend a baby otter."}'}]
Final output:
{
  "id": "resp_0b790d2a6e363c610068e0174a787481979c6adbcaed050e12",
  "created_at": 1759516490.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "Respond only with a horoscope generated by a tool.",
  "metadata": {},
  "model": "gpt-5-2025-08-07",
  "object": "response",
  "output": [
    {
      "id": "msg_0b790d2a6e363c61006